In [32]:
import psycopg2
from flask import Flask, jsonify, request, after_this_request
from flask_cors import CORS
from shapely.geometry import LineString
from shapely.geometry import Point
from shapely.ops import transform
from shapely import ops
from shapely import wkb
import pyproj

In [34]:
def clean_and_update_linestring():
    trip_id = str(46)
    
    #connect to db
    db_credentials = {"dbname": 'gta',
                  "port": 5432,
                  "user": 'gta_p8',
                  "password": 'r7sdkfdq',
                  "host": 'ikgpgis.ethz.ch'}
    conn = psycopg2.connect(**db_credentials)
    cur = conn.cursor()

    # Fetch the linestring from the database
    sql_string = "SELECT geometry FROM gta_p8.trip WHERE trip_id = "+trip_id+""
    cur.execute(sql_string)
    linestring_wkt = cur.fetchone()

    # Convert the WKT string to a Shapely LineString
    original_linestring = wkb.loads(bytes.fromhex(linestring_wkt[0]))

    clean_linestring

    conn.commit()
    conn.close()

    # Clean the linestring
    cleaned_linestring = clean_linestring(original_linestring)

    print(cleaned_linestring)

def clean_linestring(linestring, distance_threshold_meters):
    # Convert distance threshold to degrees (approximate conversion for Zurich, Switzerland)
    distance_threshold_degrees = distance_threshold_meters / 111000

    # Filter out points that are too far from their neighbors
    cleaned_coordinates = [Point(linestring.coords[0])]
    for i in range(1, len(linestring.coords) - 1):
        point_prev = Point(linestring.coords[i - 1])
        point_current = Point(linestring.coords[i])
        point_next = Point(linestring.coords[i + 1])

        if point_prev.distance(point_next) < distance_threshold_degrees:
            cleaned_coordinates.append(point_current)

    cleaned_coordinates.append(Point(linestring.coords[-1]))

    # Create a new LineString with the cleaned coordinates
    cleaned_linestring = LineString(cleaned_coordinates)
    return cleaned_linestring

# Example usage
linestring_id = 1  # Replace with the actual ID of the linestring you want to clean
clean_and_update_linestring()


AttributeError: 'tuple' object has no attribute 'distance'